In [ ]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import resample
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # To supress WavFileWarning: Chunk (non-data) not understood, skipping it.

In [ ]:
rate, data = wavfile.read('../data/audio/88_Key_Ascending_Chromatic_Scale.wav')

# Downsample from 41000 to dsamp_rate
dsamp_rate = 10250
dsamp_data = resample(data[:,0], int((dsamp_rate/rate) * data.shape[0]))

x = np.linspace(0, 90, data[:,0].shape[0], endpoint=False)
xnew = np.linspace(0, 90, dsamp_data.shape[0], endpoint=False)

print(x.shape)
print(xnew.shape)
print(data[:,0].shape)
print(dsamp_data.shape)

#plt.plot(x, data[:,0], 'go-', xnew, dsamp_data, '.-', data.shape[0], data[0,0], 'ro')
#plt.legend(['data', 'resampled'], loc='best')
plt.figure(1)
plt.plot(data[:,0])
plt.figure(2)
plt.plot(dsamp_data)
plt.show()

print(data.shape)
print(data[:,0].shape)

In [ ]:
######################################################################
# Keep FRAME_SIZE and FRAMES_PER_FFT to be powers of two.

NOTE_MIN = 21          # A0
NOTE_MAX = 108         # C8
FSAMP = rate           # Sampling frequency in Hz
FRAME_SIZE = 2097152   # Number of samples per frame
FRAMES_PER_FFT = 2     # Number of frames FFT average across (set to an overlap of 2:1)

######################################################################
# Note: as SAMPLES_PER_FFT goes up, the frequency step size decreases
# (resolution increases), however, it will incur more processing delay.

SAMPLES_PER_FFT = FRAME_SIZE*FRAMES_PER_FFT
FREQ_STEP = float(FSAMP)/SAMPLES_PER_FFT

######################################################################

NOTE_NAMES = 'C C# D D# E F F# G G# A A# B'.split()

print(SAMPLES_PER_FFT)
print(FRAME_SIZE)
print("FFT resolution:", FREQ_STEP, "Hz")
print("FFT \'window\':", (1.0/FREQ_STEP)/1000.0, "ms")
print("Frames per second: ", float(FSAMP)/FRAME_SIZE)
print(float(FSAMP)/2)

print("Power", np.power(2, 21))

# Want SAMPLES_PER_FFT = (FSAMP * 1000.0) * 128.0, thus
# (FRAME_SIZE * FRAMES_PER_FFT) = (FSAMP * 1000.0) * 128.0
print(SAMPLES_PER_FFT)
print((FSAMP * 1000.0) * 128.0)


In [ ]:
######################################################################
# These three functions are based upon this very useful webpage:
# https://newt.phys.unsw.edu.au/jw/notes.html

def freq_to_number(f): return 69 + 12*np.log2(f/440.0)
def number_to_freq(n): return 440 * 2.0**((n-69)/12.0)
def note_name(n): return NOTE_NAMES[n % 12] + str(n/12 - 1)


In [ ]:
%%time

# Get min/max FFT index for notes NOTE_MIN to NOTE_MAX. See numpy.rfftfreq()
def note_to_fftbin(n): return number_to_freq(n)/FREQ_STEP
imin = max(0, int(np.floor(note_to_fftbin(NOTE_MIN-1))))
imax = min(SAMPLES_PER_FFT, int(np.ceil(note_to_fftbin(NOTE_MAX+1))))

# Create Hanning window function
window = 0.5 * (1 - np.cos(np.linspace(0, 2*np.pi, SAMPLES_PER_FFT, False)))

print('sampling at', FSAMP, 'Hz with max resolution of', FREQ_STEP, 'Hz\n')

last_note = ''
num_frames = 0
fft_buffer = np.zeros(SAMPLES_PER_FFT, dtype=np.float32)
for fstep in range(0, data.shape[0], FRAME_SIZE):
#for fstep in range(0, dsamp_data.shape[0], FRAME_SIZE):
    
    # Shift the buffer down and new data in
    fft_buffer[:-FRAME_SIZE] = fft_buffer[FRAME_SIZE:]
    fft_buffer[-FRAME_SIZE:] = data[fstep:(FRAME_SIZE + fstep), 0]
    #fft_buffer[-FRAME_SIZE:] = dsamp_data[fstep:(FRAME_SIZE + fstep)]

    # Run the FFT on the windowed buffer
    fft = np.fft.rfft(fft_buffer * window)

    # Get frequency of maximum response in range
    freq = (np.abs(fft[imin:imax]).argmax() + imin) * FREQ_STEP

    # Get note number and nearest note
    n = freq_to_number(freq)
    n0 = int(round(n))

    # Console output once we have a full buffer
    num_frames += 1

    if (num_frames >= FRAMES_PER_FFT) and (note_name(n0) != last_note):
        print('freq: {:7.2f} Hz     note: {:>3s} {:+.2f}'.format(freq, note_name(n0), n-n0))
        last_note = note_name(n0)